# PyMongo

In [1]:
import os
import certifi
from pymongo import MongoClient, ASCENDING, DESCENDING
from bson.json_util import dumps, ObjectId
import json

In [2]:
# Utilities
from dates_management import add_created_at

In [3]:
%env MONGO_ATLAS_HOST=
%env MONGO_ATLAS_USERNAME=
%env MONGO_ATLAS_PASSWORD=

env: MONGO_ATLAS_HOST=iotaimpactcluster.vqlak.mongodb.net
env: MONGO_ATLAS_USERNAME=development
env: MONGO_ATLAS_PASSWORD=erFdT23Ds8Cdf3mawQPL


In [4]:
def get_db_handle(**kwargs):
    ''' Database connector
    Optional Arguments:
        host, username, password, db_name
    Return:
        client: Mongo Client
        db_handle: client[db_name]
    '''
    
    host = kwargs.get('host', os.environ["MONGO_ATLAS_HOST"])
    username = kwargs.get('username', os.environ["MONGO_ATLAS_USERNAME"])
    password = kwargs.get('password', os.environ["MONGO_ATLAS_PASSWORD"])
    db_name = kwargs.get('db_name', 'iotaimpact')
    
    retry = kwargs.get('retry', 0)
    try:
        url = f"mongodb+srv://{username}:{password}@{host}/{db_name}?retryWrites=true&w=majority"
        client = MongoClient(url, tlsCAFile=certifi.where())

        db_handle = client[db_name]
        return db_handle, client

    except Exception as e:
        retry += 1
        print(f'Something went wrong, attempt {retry} to re-connect to Mongodb')

        if retry < 3:
            return get_db_handle(retry=retry)
        else:
            raise Exception(f'Can not connect with Mongodb, contact your DBA.\nError: {e}')

In [5]:
def delete(**kwargs):
    ''' Delete one or all documents in a collection
    Example:
        collection="test"
        query_filter = {"test": "mongo"}
        
    Required Arguments:
        collection: target collection
        query_filter: specify a filter if you want to update just one document

    Optional Arguments:
        database credentials
        amount: "all" (default: "one")
    '''

    db_name = kwargs.get('db_name', 'iotaimpact')
    db, _ = get_db_handle(db_name=db_name)

    collection = kwargs.get('collection', None)
    query_filter = kwargs.get('query_filter', None)
    amount = kwargs.get('amount', 'one')

    if amount == 'all':
        return str(db[collection].delete_many(query_filter))
    
    else:
        return str(db[collection].delete_one(query_filter))

In [6]:
def insert(**kwargs):
    ''' General Purpose INSERT
    Arguments: 
        db_name (optional): target database (default: iotaimpact)
        collection: target collection
        document(s): Object or List of Objects to be stored
    Return: 
        ID(s) of the inserted object(s)
    '''
    db_name = kwargs.get('db_name', 'iotaimpact')
    db, _ = get_db_handle(db_name=db_name)

    collection = kwargs.get('collection', 'undefined_collection')
    document = kwargs.get('document', None)
    documents = kwargs.get('documents', None)
    
    # insert_one
    if document:
        add_created_at(document)
        return str(db[collection].insert_one(document).inserted_id)
    
    # insert_many
    elif documents:
        return str(db[collection].insert_many(documents))

In [56]:
def update(**kwargs):
    ''' Insert one or more fields into one or all documents in a collection
    Example:
        collection="test"
        document = {"new_field": 'new_value'}
        query_filter = {"test": "mongo"}
        
    Required Arguments:
        collection: target collection
        document: dictionary with the new fields
        query_filter: specify a filter if you want to update just one document

    Optional Arguments:
        database credentials
        amount: "all" (default: "one")
    '''

    db_name = kwargs.get('db_name', 'iotaimpact')
    db, _ = get_db_handle(db_name=db_name)

    query_filter = kwargs.get('query_filter', None)
    collection = kwargs.get('collection', None)
    document = kwargs.get('document', None)
    amount = kwargs.get('amount', 'one')

    if collection is None or query_filter is None or document is None:
        raise Exception('Specify collection, query_filter and document')
        
    if amount == 'all':
        # Return the count of documents updated (count: integer)
        return db[collection].update_many(query_filter, {'$set': document}).modified_count
    
    else:
        # return the updated object (type: dict)
        sort = kwargs.get('sort', 'DESC')
        sort = DESCENDING if sort=='DESC' else ASCENDING
        return db[collection].find_one_and_update(query_filter, {'$set': document}, sort=[("_id", sort)])

In [8]:
def find(**kwargs):
    ''' General Purpose FIND 
    Example:
        collection='surveys'
        query_filter={"user_token": "181.61.59.165:2021-10-26"}
        projection={"highest_match_report": 1, "paid":1, "created_at": 1}
        skip=30 (Default: 0)
        limit=50
        sort='ASC' (Default: 'DESC')
        find(collection=collection, query_filter=query_filter, projection=projection, limit=limit)
        
    Arguments: 
        - Required
        db_name (optional): target database
        collection: target collection

        - Search types (Choose One)
        id: find by ID
        query_filter: find with filter

        - Options
        projection: retrieve specified fields
        skip & limit: pagination
        sort: sort ASC or DESC by ID

    Return: 
        list of documents found
    '''
    db_name = kwargs.get('db_name', 'iotaimpact')
    db, _ = get_db_handle(db_name=db_name)

    collection = kwargs.get('collection', None)
    if collection is None:
        raise Exception('Specify the collection')

    # Find by ID
    id = kwargs.get('id', None) 
    
    # Query
    query_filter = kwargs.get('query_filter', None)
    
    # Projection
    projection = kwargs.get('projection', None)
    
    # Sort
    sort = kwargs.get('sort', 'DESC')
    sort = DESCENDING if sort=='DESC' else ASCENDING
    
    # Skip and Limit
    skip = kwargs.get('skip', 0)
    limit = kwargs.get('limit', None)
    
    search_result = None
    
    # Find by ID
    if id:
        search_result = dumps(db[collection].find_one({"_id": ObjectId(id)}, projection))

    # Find by filter
    elif query_filter: 
        
        if limit is not None:
            search_result = dumps(db[collection].find(query_filter, projection, skip=int(skip), limit=int(limit), sort=[("_id", sort)]))
        else:
            search_result = dumps(db[collection].find(query_filter, projection, skip=int(skip), sort=[("_id", sort)]))
            
    return search_result

## Insert

In [32]:
collection='surveys'
document={
    "user_token": "186.113.136.245:2021-09-28",
    "last": True
}
insert(collection=collection, document=document)

'6163206917c6725c01e9d664'

## Sort

In [156]:
collection='surveys'
query={"user_token": "186.113.136.245:2021-09-28"}
projection={"_id":0, "paid":1, "last": 1, "created_at": 1}

# Default sort (DESC)
desc_result = find(collection=collection, query=query, projection=projection)

# ASC sort
sort='ASC'
asc_result = find(collection=collection, query=query, projection=projection, sort=sort)

In [157]:
json.loads(desc_result)

[{'last': True, 'created_at': '10/10/2021 12:18:33'},
 {'last': True, 'created_at': '10/10/2021 11:57:15'},
 {'paid': False},
 {'paid': False},
 {'paid': False},
 {'paid': False},
 {'paid': False},
 {'paid': False},
 {'paid': False}]

In [158]:
json.loads(asc_result)

[{'paid': False},
 {'paid': False},
 {'paid': False},
 {'paid': False},
 {'paid': False},
 {'paid': False},
 {'paid': False},
 {'last': True, 'created_at': '10/10/2021 11:57:15'},
 {'last': True, 'created_at': '10/10/2021 12:18:33'}]

## Limit & Skip

In [150]:
collection='surveys'
query={"user_token": "186.113.136.245:2021-09-28"}
projection={"paid":1, "last": 1}
skip=1
limit=2

# find(collection=collection, query=query, projection=projection)
find(collection=collection, query=query, projection=projection, skip=skip, limit=limit)

'[{"_id": {"$oid": "61631b6b17c6725c01e9d658"}, "last": true}, {"_id": {"$oid": "615389fd687b719f52211c5a"}, "paid": false}]'

In [77]:
skip_limit

'[{"_id": {"$oid": "615381330c3e4caaa2b22c05"}, "paid": false}, {"_id": {"$oid": "615382110c3e4caaa2b22c07"}, "paid": false}]'

## Projections

In [11]:
collection='surveys'
query_filter={"user_token": "186.113.136.245:2021-09-28"}
projection={"highest_match_report":1}
# find(collection=collection, query_filter=query_filter)
result = find(collection=collection, query_filter=query_filter, projection=projection)
print(result)

[{"_id": {"$oid": "61537e1a0c3e4caaa2b22c03"}, "highest_match_report": {"title": "Labor statistics - Food industry", "relevance": "HIGH", "url": "https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/Labor+statistics+-+Food+industry.pdf"}}, {"_id": {"$oid": "615381330c3e4caaa2b22c05"}, "highest_match_report": {"title": "Soft Drink Research Report", "relevance": "HIGH", "url": "https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/Soft+Drink+Research+Report.pdf"}}, {"_id": {"$oid": "615382110c3e4caaa2b22c07"}, "highest_match_report": {"title": "Labor statistics - Food industry", "relevance": "HIGH", "url": "https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/Labor+statistics+-+Food+industry.pdf"}}, {"_id": {"$oid": "615382620c3e4caaa2b22c09"}, "highest_match_report": {"title": "US Imports", "relevance": "HIGH", "url": "https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/US+Imports.pdf"}}, {"_id": {"$oid": "615382c00c3e4caaa2b22c0b"}, "highest_match_report": {"title":

In [15]:
type(result)

str

In [18]:
# string to json
json_result = json.loads(result)
json_result

[{'_id': {'$oid': '61537e1a0c3e4caaa2b22c03'},
  'highest_match_report': {'title': 'Labor statistics - Food industry',
   'relevance': 'HIGH',
   'url': 'https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/Labor+statistics+-+Food+industry.pdf'}},
 {'_id': {'$oid': '615381330c3e4caaa2b22c05'},
  'highest_match_report': {'title': 'Soft Drink Research Report',
   'relevance': 'HIGH',
   'url': 'https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/Soft+Drink+Research+Report.pdf'}},
 {'_id': {'$oid': '615382110c3e4caaa2b22c07'},
  'highest_match_report': {'title': 'Labor statistics - Food industry',
   'relevance': 'HIGH',
   'url': 'https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/Labor+statistics+-+Food+industry.pdf'}},
 {'_id': {'$oid': '615382620c3e4caaa2b22c09'},
  'highest_match_report': {'title': 'US Imports',
   'relevance': 'HIGH',
   'url': 'https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/US+Imports.pdf'}},
 {'_id': {'$oid': '615382c00c3e4caaa2b22c0b'

In [21]:
# json to string
result_string = dumps(json_result)
result_string

'[{"_id": {"$oid": "61537e1a0c3e4caaa2b22c03"}, "highest_match_report": {"title": "Labor statistics - Food industry", "relevance": "HIGH", "url": "https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/Labor+statistics+-+Food+industry.pdf"}}, {"_id": {"$oid": "615381330c3e4caaa2b22c05"}, "highest_match_report": {"title": "Soft Drink Research Report", "relevance": "HIGH", "url": "https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/Soft+Drink+Research+Report.pdf"}}, {"_id": {"$oid": "615382110c3e4caaa2b22c07"}, "highest_match_report": {"title": "Labor statistics - Food industry", "relevance": "HIGH", "url": "https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/Labor+statistics+-+Food+industry.pdf"}}, {"_id": {"$oid": "615382620c3e4caaa2b22c09"}, "highest_match_report": {"title": "US Imports", "relevance": "HIGH", "url": "https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/US+Imports.pdf"}}, {"_id": {"$oid": "615382c00c3e4caaa2b22c0b"}, "highest_match_report": {"title"

In [22]:
type(result_string)

str

# Testing 

In [8]:
import json

### find

In [14]:
collection='dashboards'
query_filter={"user_token": "186.29.243.144:2021-10-22"}
projection={"highest_match_dashboard": 1, "_id": 1}
dashboards = json.loads(find(collection=collection, query_filter=query_filter, projection=projection))
dashboards

[{'_id': {'$oid': '617344ede4736991ca792794'},
  'highest_match_dashboard': {'title': 'Nutrition and Sandwiches',
   'relevance': 'MEDIUM',
   'url': 'https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/dashboards/nutrition+survey+crossed+sandwiches.pdf'}},
 {'_id': {'$oid': '6173425ee4736991ca792780'},
  'highest_match_dashboard': {'title': 'ConsumerExpenditures',
   'relevance': 'MEDIUM',
   'url': 'https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/dashboards/consumer+expenditures.pdf'}},
 {'_id': {'$oid': '617322cc756ee8639c6a8f08'},
  'highest_match_dashboard': {'title': 'Consumer Expenditures',
   'relevance': 'HIGH',
   'url': 'https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/dashboards/consumer+expenditures.pdf'}},
 {'_id': {'$oid': '61730901186a34752e98bf87'},
  'highest_match_dashboard': {'title': 'Consumer expenditures',
   'relevance': 'MEDIUM',
   'url': 'https://s3.us-east-1.amazonaws.com/iotatech-kendra.bucket/dashboards/consumer+expenditures.pdf'}}

### Update

In [59]:
collection="test"
query_filter = {"test": "test"}
document = {"test": 'TEST', 'TEST': 'test'}
amount='all'
result = update(collection=collection, amount=amount, query_filter=query_filter, document=document)
result

26

In [41]:
result, type(result)

('<pymongo.results.UpdateResult object at 0x0000022E2BCE0900>', str)

In [43]:
collection="test"
query_filter = {"user_id": "DELETEME"}
delete(collection=collection, query_filter=query_filter)

'<pymongo.results.DeleteResult object at 0x0000022E2BDB2A00>'

In [45]:
collection="test"
query_filter = {"user_id": "DELETEME"}
amount='all'
delete(collection=collection, amount=amount, query_filter=query_filter, document=document)

'<pymongo.results.DeleteResult object at 0x0000022E2BCF6C80>'

# Find Embedded Documents

In [14]:
collection='surveys'
query_filter={"user_token": "291020211256"}
projection={"survey": 1, "_id": 1}
surveys = json.loads(find(
    collection=collection, 
    query_filter=query_filter, 
    projection=projection,
    limit=1
))
surveys

[{'_id': {'$oid': '617d42b6af9aa8d04bf3921c'},
  'survey': {'question2': 'Marketing and sales',
   'slider1': '0',
   'slider2': '0',
   'slider3': '0',
   'slider4': '0',
   'slider5': '0',
   'slider6': '0',
   'slider7': '0',
   'slider8': '0',
   'slider9': '0',
   'slider10': '0',
   'slider11': '0',
   'slider12': '0',
   'slider13': '0',
   'slider14': '0',
   'slider15': '0',
   'slider16': '0',
   'slider17': '0',
   'slider18': '0',
   'slider19': '0',
   'slider20': '0',
   'slider21': '0',
   'slider22': '0',
   'slider23': '0',
   'slider24': '0',
   'slider25': '0',
   'slider26': '0',
   'slider27': '0',
   'slider28': '0',
   'slider29': '0',
   'slider30': '0',
   'slider31': '0',
   'slider32': '0',
   'slider33': '0',
   'slider34': '0',
   'slider35': '0',
   'slider36': '0',
   'slider37': '0',
   'slider38': '0',
   'slider39': '0',
   'slider40': '0',
   'question108': [{}],
   'slider41': '0',
   'slider42': '0',
   'slider43': '0',
   'slider44': '0',
   'quest

# Aggregations